In [1]:
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk import word_tokenize
from nltk.tag import pos_tag
import string
import matplotlib.pyplot as plt
import seaborn as sns
import re
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import numpy as np
import pandas as pd

from os import listdir

2023-12-18 08:57:23.702434: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-18 08:57:23.705195: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-18 08:57:23.742125: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 08:57:23.742161: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 08:57:23.742190: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('indian')

nltk.download('punkt')
nltk.download('words')

[nltk_data] Downloading package stopwords to /home/dai/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dai/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/dai/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/dai/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package indian to /home/dai/nltk_data...
[nltk_data]   Package indian is already up-to-date!
[nltk_data] Downloading package punkt to /home/dai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /home/dai/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
# load doc imto memory
def load_doc(filename):
    # open file in read mode
    file = open(filename,'r')
    # Read All text
    text = file.read()
    # Close the file
    file.close()
    return text

In [5]:
text = load_doc('review_polarity/txt_sentoken/pos/cv026_29325.txt')

In [6]:
text

'for those of us who weren\'t yet born when the 1960\'s rock \'n\' rolled around , monterey pop affords an affectionate glimpse of the music that influenced our parents to be hippies . \nfrom otis redding to jimi hendrix , janis joplin to the mamas and the papas , and jefferson airplane to the who , this documentary is jam-packed with contagious energy . \nbut i give fair warning that i will reveal the ending , which does not do the rest of the film the justice it deserves . \nshot in 1969 at an outdoor concert that precluded woodstock , the film defies the stereotype of the general population at the time . \nsure , some have painted their faces and smoke joints , but d . a . \npennebaker ( the war room , moon over broadway ) surprisingly chooses to show a broad spectrum of the audience . \nno matter who is watching , it all comes back to the talented musicians that stir your soul . \nthe excitement starts before the music even begins . \na young girl is cleaning thousands of seats and

In [7]:
# Turn a doc into clean tokens
def clean_doc(doc):
    # Split into tokens by white space
    tokens = doc.split()
    # Prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # Remove punctuation from each word
    tokens = [re_punc.sub('',w) for w in tokens]
    # Remove remaining token that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    #filter out short tokens
    tokens = [word for word in tokens if len(word)>1]
    return tokens

In [8]:
clean_text = clean_doc(text)
len(clean_text)

261

In [9]:
clean_text

['us',
 'werent',
 'yet',
 'born',
 'rock',
 'rolled',
 'around',
 'monterey',
 'pop',
 'affords',
 'affectionate',
 'glimpse',
 'music',
 'influenced',
 'parents',
 'hippies',
 'otis',
 'redding',
 'jimi',
 'hendrix',
 'janis',
 'joplin',
 'mamas',
 'papas',
 'jefferson',
 'airplane',
 'documentary',
 'jampacked',
 'contagious',
 'energy',
 'give',
 'fair',
 'warning',
 'reveal',
 'ending',
 'rest',
 'film',
 'justice',
 'deserves',
 'shot',
 'outdoor',
 'concert',
 'precluded',
 'woodstock',
 'film',
 'defies',
 'stereotype',
 'general',
 'population',
 'time',
 'sure',
 'painted',
 'faces',
 'smoke',
 'joints',
 'pennebaker',
 'war',
 'room',
 'moon',
 'broadway',
 'surprisingly',
 'chooses',
 'show',
 'broad',
 'spectrum',
 'audience',
 'matter',
 'watching',
 'comes',
 'back',
 'talented',
 'musicians',
 'stir',
 'soul',
 'excitement',
 'starts',
 'music',
 'even',
 'begins',
 'young',
 'girl',
 'cleaning',
 'thousands',
 'seats',
 'asked',
 'interviewer',
 'replies',
 'feels',
 '

In [10]:
# Load doc,clean amd return line of token
def doc_to_line(filename,vocab):
    # Load th edoc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # filter by vocab
    tokens = [w for w in tokens if w in vocab]
    return ' '.join(tokens)

In [12]:
vocab = open('vocab.txt')
vocab = vocab.read().split()

In [14]:
doc_to_line('review_polarity/txt_sentoken/pos/cv026_29325.txt',vocab)

'us werent yet born rock rolled around monterey pop affectionate glimpse music influenced parents hippies otis redding jimi hendrix janis joplin mamas papas jefferson airplane documentary jampacked contagious energy give fair warning reveal ending rest film justice deserves shot outdoor concert film defies stereotype general population time sure painted faces smoke joints war room moon broadway surprisingly chooses show broad spectrum audience matter watching comes back talented musicians stir soul excitement starts music even begins young girl cleaning thousands seats asked interviewer replies feels lucky moments organized craziness john phillips leader mamas papas one concert tries get touch one band tuning member remarks finally decent sound system tell watching first moments show isnt vanity playing music love appreciation street interaction audience performer continues throughout film becomes infectious audience impossible tear eyes away janis joplin belts ballad love ball chain l

In [15]:
# Load Training data
def process_train(directory,vocab):
    documents = []
    for filename in listdir(directory):
        if not filename.startswith('cv9'):
            path = directory + '/' +filename
            doc = load_doc(path)
            tokens = clean_doc(doc,vocab)
            documents.append(tokens)
    return documents

 # Load Testing data
def process_test(directory,vocab):
    documents = []
    for filename in listdir(directory):
        if filename.startswith('cv9'):
            path = directory + '/' +filename
            doc = load_doc(path)
            tokens = clean_doc(doc,vocab)
            documents.append(tokens)
    return documents


In [16]:
# Load docs in a directory
def process_docs(directory,vocab,is_train):
    documents = []
    # Walk through All the Files In The Folder
    for filename in listdir(directory):
        # Skip any reviews in test set
        if is_train and filename.startswith('cv9'):
            continue
        if not is_train and not filename.startswith('cv9'):
            continue
        # create the full path of the file to open
        path = directory + '/' + filename
        # load the doc
        doc = load_doc(path)
        # clean dDoc
        tokens = clean_doc(doc)
        # add to list
        documents.append(tokens)
    return documents


In [18]:
lines = process_docs('review_polarity/txt_sentoken/pos',vocab,False)
# Is_train = False Means Loading Testing Data

In [19]:
len(lines)

100

In [22]:
# Load and Clean a Dataset
def load_clean_dataset(vocab,is_train):
    # load documents
    neg = process_docs('review_polarity/txt_sentoken/neg',vocab,is_train)
    pos = process_docs('review_polarity/txt_sentoken/pos',vocab,is_train)
    docs = neg+pos
    # Prepare Labels
    labels = [0 for _ in range(len(neg))] + [1 for _ in range(len(pos))]
    return docs,labels


In [23]:
train,train_labels = load_clean_dataset(vocab,True)
test,test_labels = load_clean_dataset(vocab,False)


In [24]:
len(train),len(train_labels)

(1810, 1810)

In [25]:
len(test),len(test_labels)

(200, 200)

In [26]:
from keras.models import Sequential
from keras.layers import Dense,MaxPool1D,Conv1D,Embedding,Flatten

In [27]:
# define the model
def define_model(vocab_size,max_length):
    # Define Network
    model = Sequential()
    model.add(Embedding(vocab_size,100,input_length=max_length))
    model.add(Conv1D(filters=32,kernel_size=8,activation='relu'))
    model.add(MaxPool1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    # Compile Network
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics = ['accuracy'])

    # summerize Defined Model
    model.summary()
    plot_model(model,to_file='model.png',show_shapes=True)
    return model

In [28]:
define_model(10,100)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000      
                                                                 
 conv1d (Conv1D)             (None, 93, 32)            25632     
                                                                 
 max_pooling1d (MaxPooling1  (None, 46, 32)            0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 1472)              0         
                                                                 
 dense (Dense)               (None, 50)                73650     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                        

In [29]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [30]:
from keras.preprocessing.sequence import pad_sequences

In [31]:
# Integer Encode and Pad Documents
def encode_docs(tokenizer,max_length,docs):
    # Integer Encode
    encoded = tokenizer.texts_to_sequences(docs)
    # Pad Sequences
    padded = pad_sequences(encoded,maxlen=max_length,padding='post')
    return padded

In [32]:
# Create The Tokenizer
tokenizer = create_tokenizer(train)

In [33]:
X_train = tokenizer.texts_to_matrix(train,mode='binary')

In [34]:
X_train.shape

(1810, 44277)

In [35]:
X_test= tokenizer.texts_to_matrix(test,mode='binary')

In [36]:
X_test.shape

(200, 44277)

In [37]:
tokenizer.word_docs

defaultdict(int,
            {'cops': 67,
             'entirely': 102,
             'son': 201,
             'girls': 103,
             'freddie': 19,
             'imogen': 1,
             'crazy': 64,
             'bound': 43,
             'doesnt': 735,
             'youve': 143,
             'realized': 70,
             'centered': 10,
             'walking': 70,
             'popular': 145,
             'despite': 252,
             'accustomed': 8,
             'fluff': 21,
             'couple': 278,
             'resulting': 27,
             'restless': 17,
             'fantasy': 69,
             'order': 215,
             'like': 1327,
             'course': 440,
             'hit': 204,
             'comes': 527,
             'long': 534,
             'orth': 2,
             'reel': 23,
             'adorable': 18,
             'say': 525,
             'ahh': 3,
             'als': 5,
             'needy': 4,
             'cruel': 34,
             'attractive': 61,
         

In [39]:
n_words = X_train.shape[1]
model = define_model(n_words,max_length)

NameError: name 'max_length' is not defined

In [ ]:
model.fit(X_train,np.array(train_labels),epochs=10,verbose=1,batch_size=10)

In [ ]:
#!pip install pydotplus

In [ ]:
#!pip install graphviz

In [ ]:
plot_model(model,show_dtype=True,show_layer_activations=True,
           show_shapes=True,show_layer_names=True)


In [ ]:
y_pred = model.evaluate(X_test,np.array(test_labels),batch_size=1)
#Evalute function automatically calculates the loss and the Accuracy so we don't need to calculate explicitly

In [ ]:
y_pred

In [ ]:
text1 ='Best movie ever! It was great, will definitely recommend it.'
text2 = 'This is a bad Movie'

In [ ]:
def predict_sentiment(review):
    # cleanig the text
    clean_text = clean_doc(review,vocab)
    #encode and pad review
    padded=encode_docs(tokenizer,max_length,[line])
    #Predict the sentiment
    yhat=model.predict(padded,verbose=0)
    #retrieve predicted  percentage and labels
    percent_pos=yhat[0,0]
    if round(percent_pos)


In [ ]:
percentage,sentiment = pred(text1)
print(f"Review:{sentiment} Percentage:{percentage}")

In [ ]:
percentage,sentiment = pred(text2)
print(f"Review:{sentiment} Percentage:{percentage}")

In [ ]:
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())

In [ ]:
# load all Reviews
train_docs,y_train = load_clean_dataset(vocab,True)
test_docs,y_test = load_clean_dataset(vocab,False)

In [ ]:
# Create The Tokenizer
tokenizer = create_tokenizer(train_docs)

In [ ]:
# define vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size ' %vocab_size)